In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.stats import kendalltau
from scipy.stats import rankdata
from sklearn.model_selection import train_test_split
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/high'

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.csv'
pop_count = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_count_top_N_' + file_name[:-3] + 'pkl'))
pop_rating = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_rating_top_N_' + file_name[:-3] + 'pkl'))

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n_train = []
groups_n_test = []
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, 'group_' + group_sizes[i] + '.csv'), delimiter=',')
    num_groups = len(groups)
    groups_part1 =  groups[:int(num_groups*0.2)]
    groups_part2 =  groups[int(num_groups*0.2):int(num_groups*0.4)]
    groups_part3 =  groups[int(num_groups*0.4):int(num_groups*0.6)]
    groups_part4 =  groups[int(num_groups*0.6):int(num_groups*0.8)]
    groups_part5 =  groups[int(num_groups*0.8):]
    groups_train = np.concatenate((groups_part1, groups_part2, groups_part3, groups_part4))
    groups_test = groups_part5
    print(len(groups_train), len(groups_test))
    groups_n_train.append(groups_train)
    groups_n_test.append(groups_test)
    if i == 3:
        print(groups_part1, '\n\n', groups_part2, '\n\n', groups_part3, '\n\n', groups_part4, '\n\n', groups_part5, '\n\n')
        print(groups_train,'\n\n', groups_test)

66 17
33 9
20 6
14 4
[[159. 182. 148. 167. 188.]
 [184.  73. 147. 134.  41.]
 [ 96. 178.  50.  72.  33.]] 

 [[ 98. 172.   7. 196.  13.]
 [  8. 210. 116.  19. 151.]
 [ 11. 105. 174.  88.  66.]
 [ 37. 115. 124. 164.  61.]] 

 [[ 97.  52. 107. 165. 132.]
 [187. 179.  79. 211.  22.]
 [ 95.  75. 171.  74. 194.]] 

 [[190.   4.  80. 219. 130.]
 [212.  55.   3.  39. 207.]
 [ 20.  43. 180. 103.   6.]
 [ 24.  31.  57. 202. 100.]] 

 [[ 46.  47. 110. 123. 198.]
 [ 56. 191.  60. 185. 140.]
 [ 90. 126.   0.  81.  68.]
 [113.  82.  12. 199.  71.]] 


[[159. 182. 148. 167. 188.]
 [184.  73. 147. 134.  41.]
 [ 96. 178.  50.  72.  33.]
 [ 98. 172.   7. 196.  13.]
 [  8. 210. 116.  19. 151.]
 [ 11. 105. 174.  88.  66.]
 [ 37. 115. 124. 164.  61.]
 [ 97.  52. 107. 165. 132.]
 [187. 179.  79. 211.  22.]
 [ 95.  75. 171.  74. 194.]
 [190.   4.  80. 219. 130.]
 [212.  55.   3.  39. 207.]
 [ 20.  43. 180. 103.   6.]
 [ 24.  31.  57. 202. 100.]] 

 [[ 46.  47. 110. 123. 198.]
 [ 56. 191.  60. 185. 140.]
 [ 

In [4]:
for i in groups_n_train:
    print(len(i[0]))

2
3
4
5


In [5]:
svd['count'] = 1
svd

,uid,tid,rating,count
0,0.0,0.0,2.493650,1
1,0.0,6.0,2.333071,1
2,0.0,8.0,2.713340,1
3,0.0,9.0,1.949588,1
4,0.0,12.0,2.477737,1
...,...,...,...,...
11741994,219.0,54964.0,1.816855,1
11741995,219.0,54965.0,2.315346,1
11741996,219.0,54966.0,1.954432,1
11741997,219.0,54967.0,2.071366,1


In [6]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count

,tid,count,rating
166,166,1882,333.50
6310,6338,1589,92.25
17316,17377,1555,43.50
6254,6282,1396,17.50
13055,13104,1340,33.50
...,...,...,...
25819,25941,1,0.25
43440,43743,1,0.25
54130,54607,1,0.25
27486,27620,1,0.25


In [7]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized

,tid,count,rating
166,166,1882,5.000000
6310,6338,1589,4.221573
17316,17377,1555,4.131243
6254,6282,1396,3.708820
13055,13104,1340,3.560043
...,...,...,...
25819,25941,1,0.002657
43440,43743,1,0.002657
54130,54607,1,0.002657
27486,27620,1,0.002657


In [8]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [9]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [10]:
top_n_size = 20
optimal_lambdas = []
results_precision = []
results_satisfaction = []
results_tau = []
results_DCG = []
results_nDCG = []
for i in range(len(groups_n_train)):
    results_tau.append([])
    results_nDCG.append([])
    results_precision.append([])
    results_DCG.append([])
    results_satisfaction.append([])
truth_rank = []
for i in range(top_n_size):
    truth_rank.append(i+1)

In [67]:
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    for groups in groups_n_train:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    for i in range(len(groups_n_train)):
        groups = groups_n_train[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)
        
        satisfactions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            high_ratings_t = np.zeros(top_n_size)
            high_ratings_u = np.zeros(len(group))

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_ratings_t[k] += 1
                        high_ratings_u[l] += 1

            satisfaction_t = 0
            for k in range(top_n_size):
                satisfaction_t += math.log(high_ratings_t[k]+1, 10) / math.log(len(group)+1, 10)
            satisfaction_t /= top_n_size

            satisfaction_u = 0
            for k in range(len(group)):
                satisfaction_u += math.log(high_ratings_u[k]+1, 10) / math.log(top_n_size+1, 10)
            satisfaction_u /= len(group)

            satisfaction = 0.5*satisfaction_t + (1-0.5)*satisfaction_u
            satisfactions.append(satisfaction)
        
        taus = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            rating_table = pd.DataFrame() 
            for member in group:
                average = test[test['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = average
                else:
                    rating_table = rating_table.set_index('tid').add(average.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= len(group) 
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            ground_truth = []
            for _, row in rating_table.iterrows():
                ground_truth.append(row[0])
            
            pred_rank = []
            for k in top_n_list:
                if k in ground_truth:
                    pred_rank.append(ground_truth.index(k)+1)
                else:
                    pred_rank.append(top_n_size+1)
            tau, p_value = kendalltau(truth_rank, pred_rank)
            taus.append(tau)

        nDCGs = []
        DCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            rating_table = pd.DataFrame() 
            for member in group:
                average = test[test['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = average
                else:
                    rating_table = rating_table.set_index('tid').add(average.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= len(group) 
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            
            truth_rating = []
            for k in top_n_list:
                r = rating_table[rating_table['tid']==k]
                if len(r) > 0:
                    truth_rating.append(r.iloc[0]['rating'])
                else:
                    truth_rating.append(0)
            nDCG = ndcg_at_k(truth_rating, top_n_size, method=1)
            DCG = dcg_at_k(truth_rating, top_n_size, method=1)
            nDCGs.append(nDCG)
            DCGs.append(DCG)
            

    
        avg_satisfaction = 0
        for satisfaction in satisfactions:
            avg_satisfaction += satisfaction
        avg_satisfaction /= len(satisfactions)
    
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        
        n_tau = 0
        avg_tau = 0
        for tau in taus:
            if not np.isnan(tau):
                avg_tau += tau
                n_tau += 1
        avg_tau /= n_tau
        
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)      
        
        avg_DCG = 0
        for DCG in DCGs:
            avg_DCG += DCG
        avg_DCG /= len(DCGs)
        
        results_precision[i].append(avg_precision)
        results_satisfaction[i].append(avg_satisfaction)
        results_tau[i].append(avg_tau)
        results_nDCG[i].append(avg_nDCG)
        results_DCG[i].append(avg_DCG)

In [68]:
ranks_precision = []
ranks_tau = []
ranks_DCG = []
ranks_nDCG = []
ranks_satisfaction = []

In [69]:
for i in results_precision:
    ranks_precision.append(rankdata(i))
for i in results_tau:
    ranks_tau.append(rankdata(i))
for i in results_DCG:
    ranks_DCG.append(rankdata(i))
for i in results_nDCG:
    ranks_nDCG.append(rankdata(i))
for i in results_satisfaction:
    ranks_satisfaction.append(rankdata(i))

In [70]:
for i in results_precision:
    print(i)
print()
for i in results_satisfaction:
    print(i)

[0.2761194029850746, 0.27537313432835814, 0.28283582089552234, 0.3044776119402985, 0.3402985074626866, 0.3686567164179105, 0.4089552238805971, 0.43059701492537317, 0.4208955223880597, 0.36567164179104483, 0.2835820895522386]
[0.31428571428571417, 0.31249999999999994, 0.31249999999999994, 0.32678571428571435, 0.35178571428571426, 0.3732142857142858, 0.40178571428571436, 0.4321428571428571, 0.44107142857142856, 0.4285714285714285, 0.32678571428571423]
[0.359375, 0.35625, 0.359375, 0.36250000000000004, 0.378125, 0.440625, 0.459375, 0.4875, 0.49687499999999996, 0.484375, 0.39375000000000004]
[0.3541666666666666, 0.3541666666666666, 0.34583333333333327, 0.34583333333333327, 0.3499999999999999, 0.3499999999999999, 0.37916666666666665, 0.4041666666666666, 0.43333333333333335, 0.3791666666666667, 0.26249999999999996]

[0.28875842206813324, 0.2878817116177119, 0.2915875999309219, 0.30453958233364375, 0.32739095408583857, 0.3444476164300973, 0.36699559665198617, 0.37704860085961145, 0.3675380767

In [71]:
optimized_lambdas = []
for i in range(len(groups_n_test)):
    print(ranks_satisfaction[i])
#     print('--------------------------------------------------')
    print(ranks_precision[i])
#     print(ranks_tau[i])
#     print(ranks_DCG[i])
#     print(ranks_nDCG[i])
#     print(ranks_precision[i] + ranks_tau[i] + ranks_DCG[i] + ranks_nDCG[i])
#     ranks = (ranks_precision[i] + ranks_tau[i] + ranks_DCG[i] + ranks_nDCG[i]).tolist()
#     print(ranks.index(max(ranks)))
    print('--------------------------------------------------')
#     optimized_lambdas.append()

[ 3.  2.  4.  5.  7.  8.  9. 11. 10.  6.  1.]
[ 2.  1.  3.  5.  6.  8.  9. 11. 10.  7.  4.]
--------------------------------------------------
[ 4.  3.  2.  5.  6.  7.  8. 10. 11.  9.  1.]
[ 3.   1.5  1.5  5.   6.   7.   8.  10.  11.   9.   4. ]
--------------------------------------------------
[ 3.  2.  4.  5.  6.  7.  9. 10. 11.  8.  1.]
[ 2.5  1.   2.5  4.   5.   7.   8.  10.  11.   9.   6. ]
--------------------------------------------------
[ 7.5  7.5  5.5  5.5  4.   2.   9.  10.  11.   3.   1. ]
[ 6.5  6.5  2.5  2.5  4.5  4.5  8.  10.  11.   9.   1. ]
--------------------------------------------------


In [11]:
final_results_precision = []
final_results_tau = []
final_results_DCG = []
final_results_nDCG = []
for i in range(len(groups_n_test)):
    final_results_tau.append([])
    final_results_nDCG.append([])
    final_results_precision.append([])
    final_results_DCG.append([])
truth_rank = []
for i in range(top_n_size):
    truth_rank.append(i+1)

In [12]:
print(lambdas)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


In [13]:
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    for groups in groups_n_test:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    for i in range(len(groups_n_test)):
        groups = groups_n_test[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)
        
        taus = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            rating_table = pd.DataFrame() 
            for member in group:
                average = test[test['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = average
                else:
                    rating_table = rating_table.set_index('tid').add(average.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= len(group) 
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            ground_truth = []
            for _, row in rating_table.iterrows():
                ground_truth.append(row[0])
            
            pred_rank = []
            for k in top_n_list:
                if k in ground_truth:
                    pred_rank.append(ground_truth.index(k)+1)
                else:
                    pred_rank.append(top_n_size+1)
            tau, p_value = kendalltau(truth_rank, pred_rank)
            taus.append(tau)

        nDCGs = []
        DCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            rating_table = pd.DataFrame() 
            for member in group:
                average = test[test['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = average
                else:
                    rating_table = rating_table.set_index('tid').add(average.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= len(group) 
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            
            truth_rating = []
            for k in top_n_list:
                r = rating_table[rating_table['tid']==k]
                if len(r) > 0:
                    truth_rating.append(r.iloc[0]['rating'])
                else:
                    truth_rating.append(0)
            nDCG = ndcg_at_k(truth_rating, top_n_size, method=1)
            DCG = dcg_at_k(truth_rating, top_n_size, method=1)
            nDCGs.append(nDCG)
            DCGs.append(DCG)
    
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        
        n_tau = 0
        avg_tau = 0
        for tau in taus:
            if not np.isnan(tau):
                avg_tau += tau
                n_tau += 1
        if n_tau != 0:
            avg_tau /= n_tau
        else:
            avg_tau = -200 #NA
        
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)      
        
        avg_DCG = 0
        for DCG in DCGs:
            avg_DCG += DCG
        avg_DCG /= len(DCGs)
        
        final_results_precision[i].append(avg_precision)
        final_results_tau[i].append(avg_tau)
        final_results_nDCG[i].append(avg_nDCG)
        final_results_DCG[i].append(avg_DCG)

In [14]:
for i in final_results_precision:
    print(i)
print()
for i in final_results_tau:
    print(i)
print()
for i in final_results_DCG:
    print(i)    
print()
for i in final_results_nDCG:
    print(i)

[0.39117647058823524, 0.38529411764705873, 0.4029411764705882, 0.438235294117647, 0.4617647058823529, 0.511764705882353, 0.5264705882352941, 0.5441176470588236, 0.526470588235294, 0.46176470588235297, 0.3852941176470588]
[0.47777777777777775, 0.4666666666666667, 0.461111111111111, 0.46111111111111114, 0.5166666666666667, 0.5222222222222223, 0.5666666666666668, 0.577777777777778, 0.5388888888888889, 0.5166666666666667, 0.4166666666666667]
[0.4916666666666667, 0.47500000000000003, 0.4583333333333333, 0.5, 0.5166666666666667, 0.5499999999999999, 0.5499999999999999, 0.5583333333333332, 0.525, 0.4916666666666667, 0.43333333333333335]
[0.525, 0.5, 0.5, 0.525, 0.55, 0.5, 0.55, 0.5874999999999999, 0.5375, 0.5499999999999999, 0.5]

[-0.024704152426250636, 0.018333809939708016, 0.02852133009382873, 0.024736610230773646, -0.017055613117655515, 0.0012690574922314068, 0.05596135800639998, 0.12795429862609228, 0.13092090665481027, 0.05412619848167142, 0.06500514873685069]
[0.24704330580861328, 0.256